In [1]:
%load_ext autoreload
%autoreload 2

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [18]:
import dataloaders
import models
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics 
import sklearn.cluster as cluster
import numpy as np 
import random
import classifiers
from torch.autograd import Variable
import torch
import torch.nn as nn 
import torch.optim as optim
import sklearn.model_selection as model_selection
import util
import json
from collections import defaultdict
import pathlib
import pickle
from collections import defaultdict

from imblearn.under_sampling import RandomUnderSampler

In [57]:
def random_sample(X, y, ratio=0.2):
    if ratio != 1.0:
        num_elems = len(X)
        indicies = list(range(0, num_elems))
        sampled_indicies = random.sample(indicies, int(ratio * num_elems))
        sampled_indicies = np.array(sampled_indicies)
        X_sampled = X[sampled_indicies,:]
        y_sampled = y[sampled_indicies]
        return X_sampled, y_sampled
    return X, y

In [71]:
num_samples_per = 1
sample_ratios = [0.6, 0.8, 1.0]
num_splits = 5
first = 102
last = 107
house_sitting_numbers = range(first, last + 1)
input_data_path = pathlib.Path('../data/house')
delimiter = ','

In [72]:
# paramaters shared by all models
num_nodes = data.get_num_nodes()
dims = 8
epochs = 50
lr = 0.1
lr_decay=0.0
weight_decay=0.0
lam = 0.00055
p = 2

# parameters specific to SiNE
delta = 1
delta0 = 0.5
dims_array = [dims, 20, 20]
frac1 = 0.3
frac0 = 0
p0 = True if frac0 > 0 else 0

In [73]:

results = defaultdict(dict)



for house_num in [102]:
    edgelist_file = f'house-{house_num}-cleaned.csv'
    classes_file = f'house-{house_num}-classes.csv'
    
    print(f'Considering house #{house_num} dataset')
    edgelist_path = input_data_path / edgelist_file
    classes_path = input_data_path / classes_file
    data = dataloaders.SenateDataset(str(edgelist_path), str(classes_path), ratio=0.9)
    X, y = data.get_shuffled_data()
    stem = defaultdict(list)
    sine = defaultdict(list)
    results[house_num]['stem'] = stem
    results[house_num]['sine'] = sine
    for sample_ratio in sample_ratios:
        print(f'Processing {sample_ratio} of samples')
        for i in range(num_samples_per):
            
            
            X_sampled, y_sampled = random_sample(X, y, sample_ratio)
            print(f'Len of original: {len(X)}, {len(y)}')
            print(f'Len of sampled: {len(X_sampled)}, {len(y_sampled)}')
            triples, triples0 = util.triples_from_array(X_sampled, y_sampled)
            batch_size = int(frac1 * len(triples))
            batch_size0 = int(frac0 * len(triples0))
            kf = model_selection.KFold(n_splits=num_splits)
            node_names = np.arange(0, num_nodes) + 1
            node_classes = np.array(data.get_node_classes())
            node_names = node_names[node_classes < 2]
            node_classes = node_classes[node_classes < 2]
            kf.get_n_splits(node_names)
            
            
            for node_train_idx, node_test_idx in kf.split(node_names):
                node_X_train, node_y_train = node_names[node_train_idx], node_classes[node_train_idx]
                node_X_test, node_y_test = node_names[node_test_idx], node_classes[node_test_idx]
                
                # train and evaluate StEM model
                kernel_model = models.fit_pseudo_kernel_model(num_nodes, dims, X_sampled, y_sampled, epochs=epochs, p=p, 
                                              lr=lr,lr_decay=lr_decay, lam=lam, 
                            weight_decay=weight_decay, undersample=False)
                kernel_clf = linear_model.LogisticRegression()
                clf = kernel_clf
                model = kernel_model
                report = classifiers.train_and_evaluate_node_classifier(clf, model, 
                                                                        node_X_train, node_y_train, 
                                                                        node_X_test, node_y_test)
                stem[sample_ratio].append(report)
                #print(report)
                
                # train and evaluate SiNE model
                
                sine_model = models.fit_sine_model(num_nodes, dims_array, triples, triples0, delta, delta0,
                                   batch_size, batch_size0, epochs, lr=lr, lr_decay=lr_decay,
                                  lam=lam, p=p, p0=p0)
                clf_sine = linear_model.LogisticRegression()
                clf = clf_sine
                model = sine_model
                report = classifiers.train_and_evaluate_node_classifier(clf, model, 
                                                                        node_X_train, node_y_train, 
                                                                        node_X_test, node_y_test)
                sine[sample_ratio].append(report)
                
            print(results[house_num])
    

Considering house #102 dataset
Processing 0.6 of samples
Len of original: 194040, 194040
Len of sampled: 116424, 116424
The loss at epoch  1  was  0.6978643536567688
The loss at epoch  2  was  0.6822227835655212
The loss at epoch  3  was  0.6552310585975647
The loss at epoch  4  was  0.6102842092514038
The loss at epoch  5  was  0.5676258206367493
The loss at epoch  6  was  0.5489552617073059
The loss at epoch  7  was  0.5440589785575867
The loss at epoch  8  was  0.5385972857475281
The loss at epoch  9  was  0.5311555862426758
The loss at epoch  10  was  0.5156565308570862
The loss at epoch  11  was  0.4849381446838379
The loss at epoch  12  was  0.45328956842422485
The loss at epoch  13  was  0.440242737531662
The loss at epoch  14  was  0.43131735920906067
The loss at epoch  15  was  0.423547625541687
The loss at epoch  16  was  0.4191409647464752
The loss at epoch  17  was  0.41821184754371643
The loss at epoch  18  was  0.41641920804977417
The loss at epoch  19  was  0.41567775607

/Users/inzamamrahaman/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Loss at epoch  1  is  1.0068838596343994
Loss at epoch  2  is  0.9996127486228943
Loss at epoch  3  is  0.8492871522903442
Loss at epoch  4  is  1.023199200630188
Loss at epoch  5  is  0.9893762469291687
Loss at epoch  6  is  0.9183066487312317
Loss at epoch  7  is  0.8541021347045898
Loss at epoch  8  is  0.859285295009613
Loss at epoch  9  is  0.8359233140945435
Loss at epoch  10  is  0.8316099047660828
Loss at epoch  11  is  0.8467668890953064
Loss at epoch  12  is  0.895764946937561
Loss at epoch  13  is  0.8405098915100098
Loss at epoch  14  is  0.8330875039100647
Loss at epoch  15  is  0.8236872553825378
Loss at epoch  16  is  0.8193159699440002
Loss at epoch  17  is  0.8186519742012024
Loss at epoch  18  is  0.8225153684616089
Loss at epoch  19  is  0.8340077996253967
Loss at epoch  20  is  0.843255341053009
Loss at epoch  21  is  0.8221518397331238
Loss at epoch  22  is  0.8163638114929199
Loss at epoch  23  is  0.81381756067276
Loss at epoch  24  is  0.8144203424453735
Loss at

The loss at epoch  37  was  0.36356204748153687
The loss at epoch  38  was  0.36442816257476807
The loss at epoch  39  was  0.36081528663635254
The loss at epoch  40  was  0.36091458797454834
The loss at epoch  41  was  0.36001506447792053
The loss at epoch  42  was  0.3617580533027649
The loss at epoch  43  was  0.35834765434265137
The loss at epoch  44  was  0.3582405745983124
The loss at epoch  45  was  0.35636433959007263
The loss at epoch  46  was  0.35687991976737976
The loss at epoch  47  was  0.35601121187210083
The loss at epoch  48  was  0.3557267189025879
The loss at epoch  49  was  0.35421669483184814
The loss at epoch  50  was  0.3542703092098236
Loss at epoch  1  is  1.0070641040802002
Loss at epoch  2  is  0.9835840463638306
Loss at epoch  3  is  0.8900922536849976
Loss at epoch  4  is  1.0008529424667358
Loss at epoch  5  is  0.9658492803573608
Loss at epoch  6  is  0.880323052406311
Loss at epoch  7  is  0.8497695922851562
Loss at epoch  8  is  0.8670660257339478
Loss 

The loss at epoch  7  was  0.5358985662460327
The loss at epoch  8  was  0.5208243727684021
The loss at epoch  9  was  0.49007001519203186
The loss at epoch  10  was  0.4529353380203247
The loss at epoch  11  was  0.4379552900791168
The loss at epoch  12  was  0.42936620116233826
The loss at epoch  13  was  0.4217422306537628
The loss at epoch  14  was  0.41696885228157043
The loss at epoch  15  was  0.41601821780204773
The loss at epoch  16  was  0.41418740153312683
The loss at epoch  17  was  0.4137168526649475
The loss at epoch  18  was  0.41241028904914856
The loss at epoch  19  was  0.4112735092639923
The loss at epoch  20  was  0.41094693541526794
The loss at epoch  21  was  0.4093588888645172
The loss at epoch  22  was  0.4088650047779083
The loss at epoch  23  was  0.40808624029159546
The loss at epoch  24  was  0.4078060984611511
The loss at epoch  25  was  0.4070098400115967
The loss at epoch  26  was  0.40682414174079895
The loss at epoch  27  was  0.4059254229068756
The los

Loss at epoch  41  is  0.8156340718269348
Loss at epoch  42  is  0.8134844899177551
Loss at epoch  43  is  0.8141906261444092
Loss at epoch  44  is  0.8115888833999634
Loss at epoch  45  is  0.8124567866325378
Loss at epoch  46  is  0.8097101449966431
Loss at epoch  47  is  0.81083083152771
Loss at epoch  48  is  0.8088230490684509
Loss at epoch  49  is  0.8102471232414246
Loss at epoch  50  is  0.8085959553718567
The loss at epoch  1  was  0.7264767289161682
The loss at epoch  2  was  0.7146281599998474
The loss at epoch  3  was  0.7055516839027405
The loss at epoch  4  was  0.698445200920105
The loss at epoch  5  was  0.6899290680885315
The loss at epoch  6  was  0.6683814525604248
The loss at epoch  7  was  0.619205892086029
The loss at epoch  8  was  0.5704349279403687
The loss at epoch  9  was  0.5789700150489807
The loss at epoch  10  was  0.5714412331581116
The loss at epoch  11  was  0.5539996027946472
The loss at epoch  12  was  0.5566428899765015
The loss at epoch  13  was  0

Loss at epoch  25  is  0.8183651566505432
Loss at epoch  26  is  0.8158528804779053
Loss at epoch  27  is  0.8197543025016785
Loss at epoch  28  is  0.8140732645988464
Loss at epoch  29  is  0.8163769245147705
Loss at epoch  30  is  0.8112635016441345
Loss at epoch  31  is  0.8138556480407715
Loss at epoch  32  is  0.8083887100219727
Loss at epoch  33  is  0.8112186789512634
Loss at epoch  34  is  0.8093202114105225
Loss at epoch  35  is  0.8160005807876587
Loss at epoch  36  is  0.8082526326179504
Loss at epoch  37  is  0.8102450966835022
Loss at epoch  38  is  0.8090294003486633
Loss at epoch  39  is  0.8152469396591187
Loss at epoch  40  is  0.8060894012451172
Loss at epoch  41  is  0.8045992255210876
Loss at epoch  42  is  0.8021771907806396
Loss at epoch  43  is  0.8051814436912537
Loss at epoch  44  is  0.8040710091590881
Loss at epoch  45  is  0.8087501525878906
Loss at epoch  46  is  0.8046929836273193
Loss at epoch  47  is  0.8082566261291504
Loss at epoch  48  is  0.804714918

The loss at epoch  27  was  0.3646271228790283
The loss at epoch  28  was  0.3611305356025696
The loss at epoch  29  was  0.36203518509864807
The loss at epoch  30  was  0.3614310622215271
The loss at epoch  31  was  0.3630005717277527
The loss at epoch  32  was  0.3578920066356659
The loss at epoch  33  was  0.357730507850647
The loss at epoch  34  was  0.35726335644721985
The loss at epoch  35  was  0.3581041395664215
The loss at epoch  36  was  0.3569302260875702
The loss at epoch  37  was  0.3570627272129059
The loss at epoch  38  was  0.35503843426704407
The loss at epoch  39  was  0.3551991879940033
The loss at epoch  40  was  0.35477420687675476
The loss at epoch  41  was  0.35463955998420715
The loss at epoch  42  was  0.35361385345458984
The loss at epoch  43  was  0.35361504554748535
The loss at epoch  44  was  0.35318079590797424
The loss at epoch  45  was  0.3528766930103302
The loss at epoch  46  was  0.35226941108703613
The loss at epoch  47  was  0.3521858751773834
The l

The loss at epoch  13  was  0.5184506177902222
The loss at epoch  14  was  0.506072461605072
The loss at epoch  15  was  0.4859146177768707
The loss at epoch  16  was  0.4651322066783905
The loss at epoch  17  was  0.451627641916275
The loss at epoch  18  was  0.44196486473083496
The loss at epoch  19  was  0.43307530879974365
The loss at epoch  20  was  0.42689499258995056
The loss at epoch  21  was  0.42400214076042175
The loss at epoch  22  was  0.42315995693206787
The loss at epoch  23  was  0.4218340814113617
The loss at epoch  24  was  0.42098578810691833
The loss at epoch  25  was  0.42047104239463806
The loss at epoch  26  was  0.41998228430747986
The loss at epoch  27  was  0.4196426272392273
The loss at epoch  28  was  0.4191352427005768
The loss at epoch  29  was  0.4187757074832916
The loss at epoch  30  was  0.41887927055358887
The loss at epoch  31  was  0.4179835617542267
The loss at epoch  32  was  0.41774243116378784
The loss at epoch  33  was  0.4172033965587616
The l

Loss at epoch  48  is  0.8051046133041382
Loss at epoch  49  is  0.8060309290885925
Loss at epoch  50  is  0.8075053691864014
The loss at epoch  1  was  0.6755640506744385
The loss at epoch  2  was  0.6706199049949646
The loss at epoch  3  was  0.6630428433418274
The loss at epoch  4  was  0.6507737040519714
The loss at epoch  5  was  0.6057551503181458
The loss at epoch  6  was  0.5652206540107727
The loss at epoch  7  was  0.5455663800239563
The loss at epoch  8  was  0.4965139329433441
The loss at epoch  9  was  0.43100646138191223
The loss at epoch  10  was  0.41706743836402893
The loss at epoch  11  was  0.40741991996765137
The loss at epoch  12  was  0.4058331251144409
The loss at epoch  13  was  0.40719419717788696
The loss at epoch  14  was  0.4018346667289734
The loss at epoch  15  was  0.40009161829948425
The loss at epoch  16  was  0.39921072125434875
The loss at epoch  17  was  0.39797013998031616
The loss at epoch  18  was  0.39760974049568176
The loss at epoch  19  was  0

In [74]:
results

defaultdict(dict,
            {102: {'sine': defaultdict(list,
                          {0.6: [{'macro_f1': 0.98856400259909027,
                             'micro_f1': 0.98863636363636365},
                            {'macro_f1': 0.94856808883693744,
                             'micro_f1': 0.95454545454545459},
                            {'macro_f1': 0.82341137123745822,
                             'micro_f1': 0.86363636363636365},
                            {'macro_f1': 0.828125,
                             'micro_f1': 0.84090909090909094},
                            {'macro_f1': 0.84827586206896566,
                             'micro_f1': 0.86363636363636365}],
                           0.8: [{'macro_f1': 0.9770833333333333,
                             'micro_f1': 0.97727272727272729},
                            {'macro_f1': 0.94856808883693744,
                             'micro_f1': 0.95454545454545459},
                            {'macro_f1': 0.5636918382913807,
  

In [66]:
import json

In [67]:
fp = open('nclassv1.json', 'w')
json.dump(results, fp)
fp.close()

In [ ]:
d